In [12]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
pandas.set_option('display.max_colwidth', -1)

In [24]:
# load the dataset
# import urllib
# link = "https://gist.github.com/kunalj101/ad1d9c58d338e20d09ff26bcc06c4235"
# f = urllib.urlopen(link)
# myfile = f.read()
# print(myfile)


import requests

link = "https://gist.githubusercontent.com/kunalj101/ad1d9c58d338e20d09ff26bcc06c4235/raw/1d2261e2276cbb0257a2ed6e2f1f4320464c7c07/corpus"
f = requests.get(link).text

# print(f.text)

labels, texts = [], []

for i, line in enumerate(f.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(content[1:])

# Create a dataframe using texts and labels
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels
trainDF.head()

,text,label
0,"[Stuning, even, for, the, non-gamer:, This, sound, track, was, beautiful!, It, paints, the, senery, in, your, mind, so, well, I, would, recomend, it, even, to, people, who, hate, vid., game, music!, I, have, played, the, game, Chrono, Cross, but, out, of, all, of, the, games, I, have, ever, played, it, has, the, best, music!, It, backs, away, from, crude, keyboarding, and, takes, a, fresher, step, with, grate, guitars, and, soulful, orchestras., It, would, impress, anyone, who, cares, to, listen!, ^_^]",__label__2
1,"[The, best, soundtrack, ever, to, anything.:, I'm, reading, a, lot, of, reviews, saying, that, this, is, the, best, 'game, soundtrack', and, I, figured, that, I'd, write, a, review, to, disagree, a, bit., This, in, my, opinino, is, Yasunori, Mitsuda's, ultimate, masterpiece., The, music, is, timeless, and, I'm, been, listening, to, it, for, years, now, and, its, beauty, simply, refuses, to, fade.The, price, tag, on, this, is, pretty, staggering, I, must, say,, but, if, you, are, going, to, buy, any, cd, for, this, much, money,, this, is, the, only, one, that, I, feel, would, be, worth, every, penny.]",__label__2
2,"[Amazing!:, This, soundtrack, is, my, favorite, music, of, all, time,, hands, down., The, intense, sadness, of, ""Prisoners, of, Fate"", (which, means, all, the, more, if, you've, played, the, game), and, the, hope, in, ""A, Distant, Promise"", and, ""Girl, who, Stole, the, Star"", have, been, an, important, inspiration, to, me, personally, throughout, my, teen, years., The, higher, energy, tracks, like, ""Chrono, Cross, ~, Time's, Scar~"",, ""Time, of, the, Dreamwatch"",, and, ""Chronomantique"", (indefinably, remeniscent, of, Chrono, Trigger), are, all, absolutely, superb, as, well.This, soundtrack, is, amazing, music,, probably, the, best, of, this, composer's, work, (I, haven't, heard, the, Xenogears, soundtrack,, so, I, ...]",__label__2
3,"[Excellent, Soundtrack:, I, truly, like, this, soundtrack, and, I, enjoy, video, game, music., I, have, played, this, game, and, most, of, the, music, on, here, I, enjoy, and, it's, truly, relaxing, and, peaceful.On, disk, one., my, favorites, are, Scars, Of, Time,, Between, Life, and, Death,, Forest, Of, Illusion,, Fortress, of, Ancient, Dragons,, Lost, Fragment,, and, Drowned, Valley.Disk, Two:, The, Draggons,, Galdorb, -, Home,, Chronomantique,, Prisoners, of, Fate,, Gale,, and, my, girlfriend, likes, ZelbessDisk, Three:, The, best, of, the, three., Garden, Of, God,, Chronopolis,, Fates,, Jellyfish, sea,, Burning, Orphange,, Dragon's, Prayer,, Tower, Of, Stars,, Dragon, God,, and, Radical, Dreamers, -, Unstealable, ...]",__label__2
4,"[Remember,, Pull, Your, Jaw, Off, The, Floor, After, Hearing, it:, If, you've, played, the, game,, you, know, how, divine, the, music, is!, Every, single, song, tells, a, story, of, the, game,, it's, that, good!, The, greatest, songs, are, without, a, doubt,, Chrono, Cross:, Time's, Scar,, Magical, Dreamers:, The, Wind,, The, Stars,, and, the, Sea, and, Radical, Dreamers:, Unstolen, Jewel., (Translation, varies), This, music, is, perfect, if, you, ask, me,, the, best, it, can, be., Yasunori, Mitsuda, just, poured, his, heart, on, and, wrote, it, down, on, paper.]",__label__2
